# Build and visualize 3D stratigraphic model
This notebook walks through the generation of a 3D stratigraphic model of the avulsion site. The model takes an initial centerline that represents the first pre-avulsion (T1) channel position. Forward modeling this centerline through time produces the 'last' pre-avulsion (T1) channel position. The next channel opbject is the initial post-avulsion (T2) centerline, which is subsequently forward modeled to produce the 'last' post-avulsion (T2) centerline. We then use the *mayavi* library to visualize the model in 3D

### Install necessary dependencies
Uncomment the next cell to install Python libraries required for this approach. 

In [ ]:
#pip install mayavi matplotlib numpy pandas scipy blockdiagram geopandas shapely stratigraph

### Import libraries

In [ ]:
import meanderpy as mp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import blockdiagram as bd
import geopandas as gpd
from shapely.geometry import LineString
import stratigraph as sg
%matplotlib qt

### Load centerlinee data and prepare them for input to meanderpy

In [ ]:
### We were using this to make the figures
centerline_dir = '../modeling/centerlines/'
t1_shp = gpd.read_file(centerline_dir+'t1_initial_centerline.shp')

### Experimenting with a new t1 initial
#t1_shp = gpd.read_file('/Users/cole/Dropbox/Utah_Fieldwork/fluvial_modeling/cm_centerlines/dec2023_centerlines/t1_start_centerline_Jan20.shp')

line = t1_shp['geometry'].iloc[0]
x, y = line.xy
z = np.ones_like(x)
x,y,z,_,_,_,_,_ = mp.resample_centerline(x,y,z,5.0)    # resample if needed

### Flip coords
x = np.flip(x)
y = np.flip(y)

### Subtract UTM
x = x-560000
y = y-4300000

# Apply Savitzky-Golay filter
window_size = 11  # Adjust the window size as needed
order = 2  # Adjust the polynomial order as needed

x_smooth = sp.signal.savgol_filter(x, window_size, order)
y_smooth = sp.signal.savgol_filter(y, window_size, order)

### Reassign x,y to their smoothed counterpart
x = x_smooth
y = y_smooth

### Model parameters

In [ ]:
W = 50                 # channel width (m)
D = 2.0               # channel depth (m)
pad = 10
deltas = 5.0
nit = 2000            # number of iterations
depths = D * np.ones((nit,))
Cfs = 0.01 * np.ones((nit,))              # dimensionless Chezy friction factor
crdist = 2.0 * W               # threshold distance at which cutoffs occur
kl = 2/(365*24*60*60.0)   # migration rate constant (m/s)
kv =  1.0E-12               # vertical slope-dependent erosion rate constant (m/s)
dt = 2*0.05*365*24*60*60.0     # time step (s)
dens = 1000                  # density of water (kg/m3)
saved_ts = 200              # which time steps will be saved
Sl = 0.0008                     # initial slope (matters more for submarine channels than rivers)
t1 = 1                    # time step when incision starts
t2 = 1                    # time step when lateral migration starts
t3 = 1                    # time step when aggradation starts
aggr_factor = 6.34196e-10 #20 mm/yr

### Build initial channel belt object

In [ ]:
### Make first channel
ch = mp.Channel(x,y,z,W,D)

### Make ChannelBelt
chb = mp.ChannelBelt(channels=[ch], cutoffs=[], cl_times=[0.0], cutoff_times=[])

### Migrate the channel to the last pre-cutoff timestep

In [ ]:
### Migrate channel
chb.migrate(nit,saved_ts,deltas,pad,crdist,depths,Cfs,kl,kv,dt,dens,t1,t2,t3,aggr_factor) # channel migration


### Write the last pre-avulsion (T1) centerline to a shapefile

In [ ]:
# View Last channel centerline coordinates
t1_final_x = chb.channels[-1].x.copy() + 560000
t1_final_y = chb.channels[-1].y.copy() + 4300000

# Create a LineString geometry from x, y coordinates
line_geometry = LineString(zip(t1_final_x, t1_final_y))

# Create a GeoDataFrame with the LineString geometry
gdf_result = gpd.GeoDataFrame(geometry=[line_geometry])

# Specify the output shapefile path
output_shapefile_t1 = centerline_dir+'t1_final_centerline.shp'

# Save the GeoDataFrame to a new shapefile
gdf_result.to_file(output_shapefile_t1)

### Build pre-avulsion 3D fluvial model

In [ ]:
min_x = 2789.150971221104
max_x = 4802.228898997582
min_y = 2545.097408532824
max_y = 4342.6693798509295

In [ ]:
h_mud = 1.0 * np.ones((len(chb.channels),)) # thickness of overbank deposit for each time step
dx = 1 # gridcell size in meters
diff_scale = 2.0 * W/dx
v_coarse = 10.0 # deposition rate of coarse overbank sediment, in m/year (excluding times of no flooding)
v_fine = 0.0 # deposition rate of fine overbank sediment, in m/year (excluding times of no flooding)

chb_3d_preAvulsion, xmin, xmax, ymin, ymax, dists, zmaps = mp.build_3d_model(chb, 'fluvial', 
            h_mud=h_mud, h=4.0, w=W, 
            bth=0.0, dcr=10.0, dx=dx, delta_s=deltas, dt=dt, starttime=chb.cl_times[0], endtime=chb.cl_times[-1],
            diff_scale=diff_scale, v_fine=v_fine, v_coarse=v_coarse, 
            xmin=min_x-100, xmax=max_x+100, ymin=min_y-100, ymax=max_y+100)

### Import the initial post-avulsion (T2) centerline
This will not be generated automatically. 't2_initial_centerline.shp' is contained in the data repository and will be pathed automatically.

In [ ]:
t2_shp = gpd.read_file(centerline_dir+'t2_initial_centerline.shp')
line = t2_shp['geometry'].iloc[0]
x, y = line.xy
z = np.ones_like(x)
x,y,z,_,_,_,_,_ = mp.resample_centerline(x,y,z,5.0)    # resample if needed

x = x-560000
y = y-4300000

# Apply Savitzky-Golay filter
window_size = 31  # Adjust the window size as needed
order = 2  # Adjust the polynomial order as needed

x_smooth = sp.signal.savgol_filter(x, window_size, order)
y_smooth = sp.signal.savgol_filter(y, window_size, order)

## Reassign x,y to their smoothed counterpart
x = x_smooth
y = y_smooth

### Plot the the last centerline to see how many points should be held constant
Use the zoom to to inspect what node number the avulsion coincides with. Depth upstream of this node should be held constant

In [ ]:
### Plot with z values labeled to make sure things make sense
fix, ax = plt.subplots()
ax.plot(chb.channels[0].x,chb.channels[0].y,'grey',label='Pre-avulsion initial centerline')
ax.plot(chb.channels[-1].x, chb.channels[-1].y,'k-',label='Post-avulsion initial centerline')

ax.plot(x,y,'r-',label='t2_initial')
ax.plot(x,y,'k.')
for i, val in enumerate(x):
    if i % 5 == 0:
        ax.text(val, y[i], i)

plt.legend()
plt.axis('equal');

In [ ]:
def generate_z_coordinates_with_constant(x_coordinates, y_coordinates, constant_values_count, lower_bound, upper_bound):
    z_coordinates = []

    # Set the first N values to a constant
    z_coordinates.extend([upper_bound] * min(constant_values_count, len(x_coordinates)))

    for i in range(constant_values_count, len(x_coordinates)):
        # Calculate z based on distance from the first (x, y) pair
        Z = upper_bound - (i - constant_values_count) * ((upper_bound - lower_bound) / (len(x_coordinates) - constant_values_count - 1))

        # Ensure z is within the specified bounds
        Z = max(min(Z, upper_bound), lower_bound)

        z_coordinates.append(Z)
        z_coords = np.asarray(z_coordinates)
    return z_coords


### Modify z values to emulate floodplain topography
The avulsion corresponds to node 225; all values upstream are held constant and values downstream decrease linearly away from the avulsion.

In [ ]:
N = 225
z = generate_z_coordinates_with_constant(x, y, N, np.min(chb.channels[0].z), np.max(chb.channels[-1].z))

## Plot all the different centerlines to ensure they make sense

In [ ]:
### Plot with z values labeled to make sure things make sense
fix, ax = plt.subplots()
ax.plot(chb.channels[0].x,chb.channels[0].y,'grey', linestyle='--',label='Pre-avulsion initial centerline')
ax.plot(chb.channels[-1].x, chb.channels[-1].y,'k-',label='Post-avulsion initial centerline')

### Plot initial centerline configuration with elevations 
for i, val in enumerate(chb.channels[0].x):
        if i % 10 == 0:
            ax.plot(chb.channels[0].x[i], chb.channels[0].y[i],'k.')
            ax.text(chb.channels[0].x[i]+10, chb.channels[0].y[i], round(chb.channels[0].z[i], 2))

### Plot final pre-avulsion centerline with elevations
for i, val in enumerate(chb.channels[-1].x):
        if i % 10 == 0 and i > 225:
            ax.plot(chb.channels[-1].x[i], chb.channels[-1].y[i],'k.')
            ax.text(chb.channels[-1].x[i]+10, chb.channels[-1].y[i], round(chb.channels[-1].z[i], 2))

ax.plot(x,y,'g-',label='Post-avulsion initial centerline')
#ax.plot(x,y,'k.')
for i, val in enumerate(x):
    if i % 10 == 0:
        ax.plot(x[i],y[i],'k.')
        ax.text(val+10, y[i], round(z[i], 2))
plt.legend(loc='best')
plt.axis('equal')

## Initialize model for post avulsion (T2 channel) migration

In [ ]:
### Make first t2 channel
ch2 = mp.Channel(x,y,z,W,D)
chb.channels.append(ch2)

## Run Simulation for post-avulsion (T2 channel) migration

In [ ]:
chb.migrate(nit,saved_ts,deltas,pad,crdist,depths,Cfs,kl,kv,dt,dens,t1,t2,t3,aggr_factor) # channel migration
fig = chb.plot('strat', 20, 60, chb.cl_times[-1], len(chb.channels)) # plotting

### Export final T2 centerline

In [ ]:
# View Last channel centerline coordinates
t2_final_x = chb.channels[-1].x.copy() + 560000
t2_final_y = chb.channels[-1].y.copy() + 4300000

# Create a LineString geometry from x, y coordinates
line_geometry = LineString(zip(t2_final_x, t2_final_y))

# Create a GeoDataFrame with the LineString geometry
gdf_result = gpd.GeoDataFrame(geometry=[line_geometry])

# Specify the output shapefile path
output_shapefile_t2 = centerline_dir+'t2_final_centerline.shp'

# Save the GeoDataFrame to a new shapefile
gdf_result.to_file(output_shapefile_t2)

### Plot all initial and final centerlines for QC

In [ ]:
### Plot with z values labeled to make sure things make sense
fix, ax = plt.subplots()
ax.plot(chb.channels[0].x,chb.channels[0].y,'k', linestyle='--',label='Pre-avulsion initial centerline')
ax.plot(chb.channels[9].x, chb.channels[9].y,'k', linestyle='-', label='Post-avulsion final centerline')
ax.plot(chb.channels[10].x, chb.channels[10].y,'g', linestyle='--', label='Post-avulsion initial centerline')
ax.plot(chb.channels[-1].x, chb.channels[-1].y,'g', linestyle='-', label='Post-avulsion final centerline')

### Plot initial centerline configuration with elevations 
for i, val in enumerate(chb.channels[0].x):
        if i % 20 == 0:
            ax.plot(chb.channels[0].x[i], chb.channels[0].y[i],'k.')
            ax.text(chb.channels[0].x[i]+10, chb.channels[0].y[i], round(chb.channels[0].z[i], 2))

### Plot final pre-avulsion centerline with elevations
for i, val in enumerate(chb.channels[9].x):
        if i % 20 == 0 and i > 225:
            ax.plot(chb.channels[9].x[i], chb.channels[9].y[i],'k.')
            ax.text(chb.channels[9].x[i]+10, chb.channels[9].y[i], round(chb.channels[9].z[i], 2))


### Plot final pre-avulsion centerline with elevations
for i, val in enumerate(chb.channels[10].x):
        if i % 20 == 0 and i > 225:
            ax.plot(chb.channels[10].x[i], chb.channels[10].y[i],'g.')
            ax.text(chb.channels[10].x[i]+10, chb.channels[10].y[i], round(chb.channels[10].z[i], 2))


### Plot final pre-avulsion centerline with elevations
for i, val in enumerate(chb.channels[-1].x):
        if i % 20 == 0 and i > 225:
            ax.plot(chb.channels[-1].x[i], chb.channels[-1].y[i],'g.')
            ax.text(chb.channels[-1].x[i]+10, chb.channels[-1].y[i], round(chb.channels[-1].z[i], 2))

plt.legend(loc='best')
plt.axis('equal')

### Restore back to UTM coordinates

In [ ]:
# Initialize min and max values with the first channel
min_x = min(chb.channels[0].x)
max_x = max(chb.channels[0].x)
min_y = min(chb.channels[0].y)
max_y = max(chb.channels[0].y)

# Loop over the rest of the channels
for channel in chb.channels:
    min_x = min(min_x, min(channel.x))
    max_x = max(max_x, max(channel.x))
    min_y = min(min_y, min(channel.y))
    max_y = max(max_y, max(channel.y))

# Print the results
print("Absolute min x:", min_x)
print("Absolute max x:", max_x)
print("Absolute min y:", min_y)
print("Absolute max y:", max_y)

### Build post-avulsion 3D fluvial model

In [ ]:
h_mud = 1.0 * np.ones((len(chb.channels),)) # thickness of overbank deposit for each time step
dx = 1 # gridcell size in meters
diff_scale = 2.0 * W/dx
v_coarse = 10.0 # deposition rate of coarse overbank sediment, in m/year (excluding times of no flooding)
v_fine = 0.0 # deposition rate of fine overbank sediment, in m/year (excluding times of no flooding)

chb_3d_postAvulsion, xmin, xmax, ymin, ymax, dists, zmaps = mp.build_3d_model(chb, 'fluvial', 
            h_mud=h_mud, h=4.0, w=W, 
            bth=0.0, dcr=10.0, dx=dx, delta_s=deltas, dt=dt, starttime=chb.cl_times[0], endtime=chb.cl_times[-1],
            diff_scale=diff_scale, v_fine=v_fine, v_coarse=v_coarse, 
            xmin=min_x-100, xmax=max_x+100, ymin=min_y-100, ymax=max_y+100)

## Use 'stratigraph' to make 3D visualizations of the stratigraphy

In [ ]:
from mayavi import mlab
### Setting for mayavi camera angle (these settings should *not* be changed)
azimuth = -151.0187062044975
elevation = 64.25212774391869
distance = 3043.97657018737
focalpoint = np.array([927.38717861, 1166.20012347, -222.73301967])
view = mlab.view(azimuth, elevation, distance, focalpoint)

### View pre-avulsion channel belt configuration

In [ ]:
# create facies volume (the facies data that comes from meanderpy is a 1D array and we need to expand it to 3D)
facies3d = np.zeros((chb_3d_preAvulsion.strat.shape[0], chb_3d_preAvulsion.strat.shape[1], chb_3d_preAvulsion.strat.shape[2]-1))
for i in range(len(chb_3d_preAvulsion.facies)):
    facies3d[:,:,i] = chb_3d_preAvulsion.facies[i] - 1 # facies codes should start at 0

In [ ]:
### Create Block Diagram with Stratigraph
mlab.figure(bgcolor=(1,1,1)) 
mlab.view(*view)
dx = 1
ve = 5.0
color_mode='facies'
bottom=np.min(chb_3d_preAvulsion.strat) - 2

sg.create_block_diagram(chb_3d_preAvulsion.strat, dx=dx, ve=ve, color_mode=color_mode, bottom=bottom, opacity=1.0, texture=None,sea_level=None, 
                        xoffset=0, yoffset=0, scale=1, ci=None, plot_contours=None, topo_min=None, topo_max=None, plot_sides=True, 
                        plot_water=False, plot_surf=True, surf_cmap='gist_earth', kmeans_colors=None)

### View post-avulsion channel belt configuration

In [ ]:
# create facies volume (the facies data that comes from meanderpy is a 1D array and we need to expand it to 3D)
facies3d = np.zeros((chb_3d_postAvulsion.strat.shape[0], chb_3d_postAvulsion.strat.shape[1], chb_3d_postAvulsion.strat.shape[2]-1))
for i in range(len(chb_3d_postAvulsion.facies)):
    facies3d[:,:,i] = chb_3d_postAvulsion.facies[i] - 1 # facies codes should start at 0

In [ ]:
### Create Block Diagram with Stratigraph
mlab.figure(bgcolor=(1,1,1)) 
mlab.view(*view)
dx = 1
ve = 5.0
color_mode='facies'
bottom=np.min(chb_3d_postAvulsion.strat) - 2

sg.create_block_diagram(chb_3d_postAvulsion.strat, dx=dx, ve=ve, color_mode=color_mode, bottom=bottom, opacity=1.0, texture=None,sea_level=None, 
                        xoffset=0, yoffset=0, scale=1, ci=None, plot_contours=None, topo_min=None, topo_max=None, plot_sides=True, 
                        plot_water=False, plot_surf=True, surf_cmap='gist_earth', kmeans_colors=None)

### Define cookies
Use the "cookies" defined by their xcoords and ycoords below to get the transects shown in manuscript figure 4

In [ ]:
### Define manually
#xcoords, ycoords = sg.select_random_section(chb_3d_postAvulsion.strat) # define x and y coordinates for random section

In [ ]:
### Pre avulsion cookie
xcoords = [959.7740262381777,
 864.727143292993,
 787.9585070680363,
 791.6141564120818,
 960.7740262381777]
ycoords = [976.5738330112887,
 801.1026644971017,
 808.4139631851929,
 1024.0972744838812,
 980.5738330112887]

In [ ]:
### Create random cookie through pre-avulsion strat
colors = [[0.9,0.9,0],[0.5,0.25,0]] # sand and mud facies (~ point bar and levee)
mlab.figure(bgcolor=(1,1,1)) 
sg.create_random_cookie(chb_3d_postAvulsion.strat,facies=facies3d,topo=chb_3d_postAvulsion.topo,scale=1,ve=3,color_mode='facies',colors=colors,
                        colormap='gist_earth',x1=xcoords[:-1],x2=xcoords[1:],y1=ycoords[:-1],y2=ycoords[1:],
                        dx=1.0,bottom=bottom,opacity=1.0)

In [ ]:
### Post avulsion cookie
xcoords = [396.80402725516126,
 327.3466897182957,
 484.53961151225474,
 543.0300010169838,
 395.80402725516126,]

ycoords = [1148.3893521814302,
 1071.6207159564733,
 932.7060408827418,
 991.196430387471,
 1151.0450015254758]

In [ ]:
### Create random cookie through post-avulsion strat
colors = [[0.9,0.9,0],[0.5,0.25,0]] # sand and mud facies (~ point bar and levee)
mlab.figure(bgcolor=(1,1,1)) 
sg.create_random_cookie(chb_3d_postAvulsion.strat,facies=facies3d,topo=chb_3d_postAvulsion.topo,scale=1,ve=3,color_mode='facies',colors=colors,
                        colormap='gist_earth',x1=xcoords[:-1],x2=xcoords[1:],y1=ycoords[:-1],y2=ycoords[1:],
                        dx=1.0,bottom=bottom,opacity=1.0)